## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,2020-11-28 17:00:34,-33.65,115.33,48.00,91,69,7.99,broken clouds
1,1,Sisimiut,GL,2020-11-28 17:03:05,66.94,-53.67,18.50,85,73,11.86,broken clouds
2,2,Thompson,CA,2020-11-28 17:03:06,55.74,-97.86,12.20,92,90,2.24,light snow
3,3,Ribeira Grande,PT,2020-11-28 17:03:06,38.52,-28.70,57.20,62,40,4.70,scattered clouds
4,4,Butaritari,KI,2020-11-28 17:03:06,3.07,172.79,81.72,81,21,17.69,few clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temp_selection_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
temp_selection_df.head(202)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Butaritari,KI,2020-11-28 17:03:06,3.07,172.79,81.72,81,21,17.69,few clouds
7,7,Atuona,PF,2020-11-28 17:03:06,-9.80,-139.03,79.20,72,1,20.89,clear sky
9,9,Rikitea,PF,2020-11-28 17:03:07,-23.12,-134.97,76.39,83,98,13.04,overcast clouds
10,10,Bonthe,SL,2020-11-28 17:03:07,7.53,-12.51,83.46,70,19,7.92,light rain
17,17,Puerto Ayora,EC,2020-11-28 17:03:07,-0.74,-90.35,77.00,73,75,14.99,broken clouds
...,...,...,...,...,...,...,...,...,...,...,...
676,676,Gibara,CU,2020-11-28 17:04:26,21.11,-76.13,82.40,69,75,9.17,broken clouds
677,677,Santa Isabel Do Rio Negro,BR,2020-11-28 17:04:26,-0.41,-65.02,84.63,82,83,6.40,broken clouds
680,680,Mataram,ID,2020-11-28 17:04:26,-8.58,116.12,78.80,94,75,2.91,broken clouds
681,681,Mananjary,MG,2020-11-28 17:04:26,-21.22,48.33,76.82,85,35,8.59,scattered clouds


In [12]:
output_data_file = "weather_data/check.csv"
temp_selection_df.to_csv(output_data_file, index_label="City_ID")

In [7]:
# 4a. Determine if there are any empty rows.
temp_selection_df.count()

City_ID                202
City                   202
Country                199
Date                   202
Lat                    202
Lng                    202
Max Temp               202
Humidity               202
Cloudiness             202
Wind Speed             202
Current Description    202
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = temp_selection_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Butaritari,KI,81.72,few clouds,3.07,172.79,
7,Atuona,PF,79.20,clear sky,-9.80,-139.03,
9,Rikitea,PF,76.39,overcast clouds,-23.12,-134.97,
10,Bonthe,SL,83.46,light rain,7.53,-12.51,
17,Puerto Ayora,EC,77.00,broken clouds,-0.74,-90.35,
20,Iaciara,BR,85.51,scattered clouds,-14.10,-46.63,
35,Arraial Do Cabo,BR,84.20,few clouds,-22.97,-42.03,
43,Hithadhoo,MV,81.91,broken clouds,-0.60,73.08,
46,Sorong,ID,80.26,broken clouds,-0.88,131.25,
47,Catuday,PH,79.07,moderate rain,16.29,119.81,


In [35]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()  
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [36]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Butaritari,KI,81.72,few clouds,3.07,172.79,Isles Sunset Lodge
7,Atuona,PF,79.20,clear sky,-9.80,-139.03,Villa Enata
9,Rikitea,PF,76.39,overcast clouds,-23.12,-134.97,Pension Maro'i
10,Bonthe,SL,83.46,light rain,7.53,-12.51,Bonthe Holiday Village
17,Puerto Ayora,EC,77.00,broken clouds,-0.74,-90.35,Finch Bay Galapagos Hotel


In [9]:

# 7. Drop the rows where there is no Hotel Name.
# hotel_df = hotel_df.dropna()
# hotel_df
hotel_df.count()

City                   202
Country                199
Max Temp               202
Current Description    202
Lat                    202
Lng                    202
Hotel Name             202
dtype: int64

In [39]:
# 8a. Create the output File (CSV)
output_data_file = "weather_data/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [10]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))